In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
os.getcwd()
print(os.getcwd())

#################Data Prep and Combined#####################
#Nielsen Info (Sales, units,ACV(dist), Median base price, final base price, tpr & SI)
#Retailer Info (Retailer Units-from raw data and Apportion)
#Price Monitoring Info (Price, median base price)
#Promo Info (Promo Period, promo days, discount)
#Catalog & Hyperocm Info
#Catalog
#Holiday Info
#Nielsen PPG-Competitors Picre Info

#path="C:\\Users\\omprakash.bankol\\Desktop\MARS_Russia\\Dataset Preparation\\"
path="..\\Process"


C:\Users\omprakash.bankol\Desktop\MARS_Russia\Analytical Dataset Creation\Code


In [2]:
### Reading apportioned weekly sales data along with Nielsen info
sales_data =pd.read_excel(path+"\\Analytical_Output_Optimizer\\Nielsen_POS_Data_Apportioned_GUM.xlsx")
sales_data=sales_data.loc[sales_data['Year']> 2017,]
sales_data.reset_index(drop=True,inplace=True)


#Drop uncommon PPG part data (SKU)
sales_data['PPG_Part_Common']=sales_data['PPG_Part'].apply(lambda x: not("Out_Any" in x) )
sales_data =sales_data.loc[sales_data['PPG_Part_Common'],]
sales_data.reset_index(drop=True,inplace=True)

#Nielsen updated units
sales_data['PPG_Part_in_Magnit']=sales_data['PPG_Part'].apply(lambda x: ("In_Magnit" in x) )
var="Nielsen_Updated_Units"
sales_data.loc[sales_data['PPG_Part_in_Magnit'],var] = sales_data.loc[sales_data['PPG_Part_in_Magnit'],'Nielsen_Units']


#rolled up
col_group=['Manufacturer', 'Brand', 'PPG', 'Date', 'Year', 'Month', 'Week_no']
sales_data_sum=sales_data.groupby(col_group)['Nielsen_Units', 'Nielsen_Sales', 'Nielsen_Volume','Nielsen_Updated_Units',
                  'X5_Units_Weekly','Magnit_Units_Weekly','X5_Units_Monthly', 'X5_LSV_Monthly', 'Magnit_Units_Monthly',
                  'Magnit_Sales_Monthly', 'Magnit_LSV_Monthly'].sum().reset_index()

sales_data_max = sales_data.groupby(col_group)['Nielsen_Numeric_dist', 'Nielsen_Weighted_dist'].max().reset_index()
sales_data0 = pd.merge(sales_data_sum,sales_data_max,on=col_group,how='left')

del sales_data
sales_data=sales_data0.copy()


#wide to long
Retailer_sales_data =sales_data[['Date','PPG','Magnit_Units_Weekly','X5_Units_Weekly']]
Retailer_sales_data_long=pd.melt(Retailer_sales_data,id_vars=['Date','PPG'],var_name="Retailer",value_name='Retailer_Units')
Retailer_sales_data_long['Retailer']=Retailer_sales_data_long['Retailer'].map({'Magnit_Units_Weekly': 'Magnit', 'X5_Units_Weekly': 'X5'})

col_drop=['Magnit_Units_Weekly','X5_Units_Weekly','X5_Units_Monthly', 'X5_LSV_Monthly','Magnit_Units_Monthly', 'Magnit_Sales_Monthly', 'Magnit_LSV_Monthly']
sales_data.drop(col_drop,axis=1,inplace=True)

#Merge Nielsen and POS data
sales_data1 = pd.merge(sales_data,Retailer_sales_data_long,on=['PPG','Date'],how='left')


#reorder
move_col = sales_data1['Retailer']
sales_data1.drop(labels=['Retailer'], axis=1, inplace = True)
sales_data1.insert(2, 'Retailer', move_col)





######Nielsen Median Base Price Calculation######
sales_data1.sort_values(['Manufacturer','Retailer','PPG','Date'], axis = 0, ascending = True,inplace = True)
sales_data1.reset_index(drop=True,inplace=True)
sales_data1['Nielsen_price_per_unit']=sales_data1['Nielsen_Sales']/sales_data1['Nielsen_Units']

for i in range(1,8):
    sales_data1['Nielsen_price_per_unit'+'_lag'+str(i)]=sales_data1.groupby(['Retailer','PPG'])['Nielsen_price_per_unit'].shift(periods=i)

#max of previous 7 weeks
Cols_Nielsen = [i for i in list(sales_data1.columns) if "Nielsen_price_per_unit_lag" in i] 
sales_data1['Nielsen_max_price_prev']=  sales_data1[Cols_Nielsen].max(axis=1)  


#sudden jump in price will map to NA
for i in range(1,8):
    var='Nielsen_price_per_unit'+'_lag'+str(i)
    sales_data1.loc[np.abs(sales_data1[var]-sales_data1['Nielsen_max_price_prev'])/sales_data1['Nielsen_max_price_prev'] > 0.05,var]=np.nan


#median base price
sales_data1['Nielsen_median_base_price']=sales_data1[Cols_Nielsen].median(axis=1) 
sales_data1['Nielsen_median_base_price']=sales_data1['Nielsen_median_base_price']

var='Nielsen_median_base_price'
idx = np.where(np.abs(sales_data1[var]-sales_data1['Nielsen_price_per_unit'])/sales_data1[var] <= 0.05, 1, 0)
sales_data1.loc[idx==1,var]=sales_data1.loc[idx==1,'Nielsen_price_per_unit']
sales_data1.loc[idx==0,var]=sales_data1.loc[idx==0,var]


#final base price (more smoother than median base)
sales_data1.sort_values(['Retailer','PPG','Date'], axis = 0, ascending = True,inplace = True)
sales_data1.index=sales_data1['Date']
final_base_price=sales_data1.groupby(['Retailer','PPG'])[var].rolling(window=26,min_periods=1).max()
final_base_price=pd.DataFrame(final_base_price)
(final_base_price).reset_index(inplace=True)
final_base_price.rename(columns={'Nielsen_median_base_price':'Nielsen_final_base_price'},inplace=True)
sales_data1.reset_index(inplace=True,drop=True)
sales_data1=pd.merge(sales_data1,final_base_price,on=['Retailer','PPG','Date'],how='left')

#na in final base price, replace by asp
idx = np.where(sales_data1['Nielsen_final_base_price'].isna(), 1, 0)
sales_data1.loc[idx==1,'Nielsen_final_base_price']=sales_data1.loc[idx==1,'Nielsen_price_per_unit']


#
var='Nielsen_final_base_price'
idx = np.where(np.abs(sales_data1[var]-sales_data1['Nielsen_median_base_price'])/sales_data1[var] >= 0.05, 1, 0)
sales_data1.loc[idx==1,'Nielsen_median_base_price']=sales_data1.loc[idx==1,var]*0.95
#sales_data1.loc[idx==0,'Nielsen_median_base_price']=sales_data1.loc[idx==0,'Nielsen_median_base_price']
sales_data2=sales_data1.copy()
Cols_Nielsen = [i for i in list(sales_data1.columns) if "Nielsen_price_per_unit_lag" in i] 
Cols_Nielsen=Cols_Nielsen+['Nielsen_max_price_prev']
sales_data2.drop(Cols_Nielsen,axis=1,inplace=True)
sales_data2.shape




(468, 18)

In [3]:
####Nielsen Combined() TPR calculation#####
sales_data2['Nielsen_TPR']=0
var='Nielsen_median_base_price'
idx = np.where((sales_data2[var]-sales_data2['Nielsen_price_per_unit'])/sales_data2[var] > 0.05, 1, 0)
sales_data2.loc[idx==1,'Nielsen_TPR']=(sales_data2[var]-sales_data2['Nielsen_price_per_unit'])/sales_data2[var]
sales_data2['Nielsen_TPR']=sales_data2['Nielsen_TPR']*100
#s


sales_data2['PPG'].value_counts()
sales_data2['Year'].value_counts()
sales_data2['Retailer'].value_counts()


Magnit    234
X5        234
Name: Retailer, dtype: int64

In [4]:
#####SI merge (calculated on Segment)#####
SI_data =pd.read_excel(path+"\\SI_Index\\GUM_17_18 Seasonality Index.xlsx")
SI_data['week_number'] ="W"+SI_data['week_number'].astype(str)
SI_data.rename(columns={'SI':'SI_17_18','week_number':'Week'},inplace=True)
sales_data2.rename(columns={'Week_no':'Week'},inplace=True)
sales_data2 = pd.merge(sales_data2,SI_data,on="Week",how="left")

SI_data =pd.read_excel(path+"\\SI_Index\\GUM_18_19 Seasonality Index.xlsx")
SI_data['week_number'] ="W"+SI_data['week_number'].astype(str)
SI_data.rename(columns={'SI':'SI_18_19','week_number':'Week'},inplace=True)
sales_data2.rename(columns={'Week_no':'Week'},inplace=True)
sales_data2 = pd.merge(sales_data2,SI_data,on="Week",how="left")

SI_data =pd.read_excel(path+"\\SI_Index\\GUM_17_18_19 Seasonality Index.xlsx")
SI_data['week_number'] ="W"+SI_data['week_number'].astype(str)
SI_data.rename(columns={'SI':'SI_17_18_19','week_number':'Week'},inplace=True)
sales_data2.rename(columns={'Week_no':'Week'},inplace=True)
sales_data2 = pd.merge(sales_data2,SI_data,on="Week",how="left")
sales_data2.shape



(468, 22)

In [5]:
sales_data2.head()

,Manufacturer,Brand,Retailer,PPG,Date,Year,Month,Week,Nielsen_Units,Nielsen_Sales,...,Nielsen_Numeric_dist,Nielsen_Weighted_dist,Retailer_Units,Nielsen_price_per_unit,Nielsen_median_base_price,Nielsen_final_base_price,Nielsen_TPR,SI_17_18,SI_18_19,SI_17_18_19
0,MARS,ORBIT,Magnit,ORBIT OTC,2017-12-31,2018,1,W1,1490000,38099800,...,94.7,20.1,4.484883e+05,25.570336,NaN,25.570336,0.000000,0.804226,0.702187,0.840807
1,MARS,ORBIT,Magnit,ORBIT OTC,2018-01-07,2018,1,W2,1741600,44582500,...,95.0,15.2,4.364255e+05,25.598588,25.598588,25.598588,0.000000,0.895066,0.788304,0.893200
2,MARS,ORBIT,Magnit,ORBIT OTC,2018-01-14,2018,1,W3,1930100,47782500,...,95.1,45.6,8.709934e+05,24.756489,24.756489,25.598588,0.000000,0.984917,0.862251,1.006442
3,MARS,ORBIT,Magnit,ORBIT OTC,2018-01-21,2018,1,W4,2347200,52925700,...,95.4,51.5,1.243014e+06,22.548441,25.570336,25.598588,11.817971,1.009257,0.888574,1.026770
4,MARS,ORBIT,Magnit,ORBIT OTC,2018-01-28,2018,1,W5,3341500,67022400,...,95.7,64.6,1.726902e+06,20.057579,25.570336,25.598588,21.559188,1.101249,0.980144,1.075275


In [6]:
#####Price Monitoring#####
#APM=Average price monitoring, WPM=Weighted Price Monitoring
price_monitor =pd.read_excel(path+"\\Price_Monitoring\\Price_Monitoring_PPG_Data.xlsx")
price_monitor.columns
price_monitor.drop(['Nielsen_Units', 'PricexUnit'],axis=1,inplace=True)
sales_data2 = pd.merge(sales_data2,price_monitor,on=['Retailer','PPG','Date'],how="left")
sales_data2.shape

(468, 28)

In [7]:
#######Promo Information##########
promo_data =pd.read_excel(path+"\\Promo\\Weekly_Promo_Gum_Data(PPG_Level).xlsx")
promo_data.rename(columns={'Week.Start.date':'Date','PPG Name':'PPG'},inplace=True)
promo_data['Retailer'].value_counts()
#promo_data['Retailer']=promo_data['Retailer'].map({'Магнит':'Magnit','Пятерочка':'X5'})
promo_data['PPG'].value_counts()
col_del=['Year','Week','Month_Name','Month']
promo_data.drop(col_del,axis=1,inplace=True)
sales_data2=pd.merge(sales_data2,promo_data,on=['Retailer','PPG','Date'],how='left')
print(sales_data2.shape)

(468, 40)


In [8]:
sales_data2.isnull().sum()

Manufacturer                   0
Brand                          0
Retailer                       0
PPG                            0
Date                           0
Year                           0
Month                          0
Week                           0
Nielsen_Units                  0
Nielsen_Sales                  0
Nielsen_Volume                 0
Nielsen_Updated_Units          0
Nielsen_Numeric_dist           0
Nielsen_Weighted_dist          0
Retailer_Units                 0
Nielsen_price_per_unit         0
Nielsen_median_base_price      4
Nielsen_final_base_price       0
Nielsen_TPR                    0
SI_17_18                       0
SI_18_19                       0
SI_17_18_19                    0
Avg_Price_Monitoring         282
Weighted_Price_Monitoring    282
WPM_median_base_price        284
WPM_final_base_price         282
APM_median_base_price        284
APM_final_base_price         282
Date start promo             377
Date end promo               377
Promo_days

In [9]:
sales_data2.columns

Index(['Manufacturer', 'Brand', 'Retailer', 'PPG', 'Date', 'Year', 'Month',
       'Week', 'Nielsen_Units', 'Nielsen_Sales', 'Nielsen_Volume',
       'Nielsen_Updated_Units', 'Nielsen_Numeric_dist',
       'Nielsen_Weighted_dist', 'Retailer_Units', 'Nielsen_price_per_unit',
       'Nielsen_median_base_price', 'Nielsen_final_base_price', 'Nielsen_TPR',
       'SI_17_18', 'SI_18_19', 'SI_17_18_19', 'Avg_Price_Monitoring',
       'Weighted_Price_Monitoring', 'WPM_median_base_price',
       'WPM_final_base_price', 'APM_median_base_price', 'APM_final_base_price',
       'Date start promo', 'Date end promo', 'Promo_days', 'Level',
       'Activity.name', 'Mechanic', 'Discount..NRV..', 'Auto.Status.Promo',
       'Manual.Status.Promo', 'Approved', 'Fact_Flag', 'Remark'],
      dtype='object')

In [10]:
#######Catalog Information###########
#is promo days same as promo_data, take PPG names
catalog_data =pd.read_excel(path+"\\HypercomAndDisplay\\Weekly_Promo_hypercom_Gum_Data(PPG_Level).xlsx")
catalog_data.rename(columns={'Seller':'Retailer','Promo Groups':'PPG'},inplace=True)
catalog_data['Retailer'].value_counts()
catalog_data['PPG'].value_counts()
catalog_data.drop(['Year','Week', 'Month_Name', 'Month','promo_days'],axis=1,inplace=True)
sales_data2=pd.merge(sales_data2,catalog_data,on=['Retailer','PPG','Date'],how='left')
print(sales_data2.shape)


#perfer hypercom dicount,
sales_data2['Promo_Hyper_Discount']=sales_data2['Avg_Discount.depth']
idx = np.where(sales_data2['Promo_Hyper_Discount'].isna(), 1, 0)
sales_data2.loc[idx==1,'Promo_Hyper_Discount']=sales_data2.loc[idx==1,'Discount..NRV..']


########Display Info############
#is promo period same as we got in promo_data?
display_data=pd.read_excel(path+"\\HypercomAndDisplay\\Disply_Output_V1.xlsx")
display_data.rename(columns={'Week.Start.date':'Date','Promo_Group':'PPG','MWC Chain':'Retailer'},inplace=True)
display_data['Retailer'].value_counts()
display_data['Retailer']=display_data['Retailer'].map({'Магнит':'Magnit','Пятерочка':'X5'})
display_data['PPG'].value_counts()
col_req=['Date','BAHS','AHS', 'miniBAHS', 'miniAHS', 'PPG', 'Retailer']
sales_data2=pd.merge(sales_data2,display_data[col_req],on=['Retailer','PPG','Date'],how='left')
print(sales_data2.shape)



(468, 54)
(468, 59)


In [11]:
#Delete cols
col_del=['sum_..Directory.distribution','Avg_..Directory.distribution','sum_Distribution.Promo.','Avg_Distribution.Promo.']
sales_data2.drop(col_del,axis=1,inplace=True)



In [12]:
#####Holdiay Flag Retailer#######
magnit_holiday=pd.read_excel(path+"\\Holiday\\Magnit Holiday Flag 18,19,20.xlsx")
x5_holiday=pd.read_excel(path+"\\Holiday\\X5 Holiday Flag 18,19,20.xlsx")
magnit_holiday['Retailer']="Magnit"
x5_holiday["Retailer"]= "X5"
pos_holiday =pd.concat([x5_holiday,magnit_holiday], axis=0, ignore_index=True,sort=True)
pos_holiday.rename(columns={'Week Start Date':'Date'},inplace=True)
pos_holiday.drop(['Year','Week End Date','Week Number'],axis=1,inplace=True)
pos_holiday.shape
sales_data3=pd.merge(sales_data2,pos_holiday,on=['Retailer','Date'],how='left')
print(sales_data3.shape)





(468, 82)


In [13]:
######## Nielsen PPG Competitors Information ########
N_PPG_comp =pd.read_excel(path+"\\Competitor\\Nielsen_PPG_Competitors_Data_GUM.xlsx")
N_PPG_comp.columns
sales_data3 = pd.merge(sales_data3,N_PPG_comp,on=['PPG','Date'],how="left")


print(sales_data3.shape)


(468, 94)


In [14]:
#remove time stamp
sales_data3['Date'] = pd.to_datetime(sales_data3['Date']).dt.date
sales_data3.isnull().sum()

Manufacturer                               0
Brand                                      0
Retailer                                   0
PPG                                        0
Date                                       0
Year                                       0
Month                                      0
Week                                       0
Nielsen_Units                              0
Nielsen_Sales                              0
Nielsen_Volume                             0
Nielsen_Updated_Units                      0
Nielsen_Numeric_dist                       0
Nielsen_Weighted_dist                      0
Retailer_Units                             0
Nielsen_price_per_unit                     0
Nielsen_median_base_price                  4
Nielsen_final_base_price                   0
Nielsen_TPR                                0
SI_17_18                                   0
SI_18_19                                   0
SI_17_18_19                                0
Avg_Price_

In [15]:
#export
sales_data3.to_excel("..\\Output\\Data_Prep_Analytical\\Analytical_Combined_Dataset_GUM_test.xlsx",index=False)
